In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!ls drive/MyDrive/OSNA\ Project\ 2/

 epoch_5_model_bert_basic	      Model_2_losses_10_epochs.h5
'epoch_6_bert_with_2_losses (1).h5'   Model_Basic.ipynb
 epoch_6_bert_with_2_losses.h5	      Model_with_2_losses.ipynb
 labelled_data.csv		      weak_data.csv


In [ ]:
import pandas as pd

df = pd.read_csv("./drive/MyDrive/OSNA Project 2/labelled_data.csv", usecols=['id', 'text', 'user', 'user_verified',
       'user_followers_count', 'user_friends_count', 'retweet_count',
       'fav_count', 'hashtags', 'target', 'label', 'snorkel_label', 'snorkel_prob'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4210 entries, 0 to 4209
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    4210 non-null   int64  
 1   text                  4210 non-null   object 
 2   user                  4210 non-null   object 
 3   user_verified         4210 non-null   bool   
 4   user_followers_count  4210 non-null   float64
 5   user_friends_count    4210 non-null   float64
 6   retweet_count         4210 non-null   float64
 7   fav_count             4210 non-null   int64  
 8   hashtags              4210 non-null   object 
 9   target                4210 non-null   object 
 10  label                 4210 non-null   float64
 11  snorkel_label         4210 non-null   float64
 12  snorkel_prob          4210 non-null   float64
dtypes: bool(1), float64(6), int64(2), object(4)
memory usage: 398.9+ KB


In [ ]:
df.head()

,id,text,user,user_verified,user_followers_count,user_friends_count,retweet_count,fav_count,hashtags,target,label,snorkel_label,snorkel_prob
0,572330946705289216,RT @Erin_Reimer: In what world would you not b...,G_melb,False,578.0,133.0,2.0,0,"[{'text': 'MKR', 'indices': [72, 76]}]",none,0.0,1.0,0.549079
1,599286108380270593,pretty much. his editor should be piiiissed at...,randileeharper,True,19435.0,766.0,0.0,9,[],neither,0.0,0.0,0.162369
2,563831508507820032,.@scarmichael18 @shaner38 He gets to legally d...,MT8_9,False,2013.0,2074.0,0.0,0,"[{'text': 'choice', 'indices': [125, 132]}]",sexism,1.0,0.0,0.389828
3,598371898938949633,@Jackson13w @lukemckinney my last tweet to you...,randileeharper,True,19435.0,766.0,0.0,0,[],neither,0.0,0.0,0.162369
4,387967465843470336,"RT @maullyolly Call me sexist, but women shoul...",YesYoureSexist,False,21476.0,219.0,0.0,0,[],sexism,1.0,1.0,0.549079


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 12.4MB/s 
     |████████████████████████████████| 1.1MB 55.6MB/s 
     |████████████████████████████████| 890kB 59.4MB/s 
     |████████████████████████████████| 2.9MB 56.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1ca6e6861f56924399af9ae78e460f29b83da3c1ce4816e8f45b133de96305db
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
vocab = set()

def process_text(text):
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    vocab.update(set(tokens))
    return tokens

In [ ]:
process_text("this is a sample textss")

['this', 'is', 'a', 'sample', 'texts', '##s']

In [ ]:
tokenizer.encode("this is a sample textss")

[101, 2023, 2003, 1037, 7099, 6981, 2015, 102]

In [ ]:
targets = ['racism', 'sexism', 'neither']
df['processed_target'] = df.target.apply(lambda x: x.split()[0])
df['processed_target'] = df.processed_target.apply(lambda x_: 'neither' if x_ == 'none' else x_)
df = df[df['processed_target'].isin(targets)]
label_map = {
    "racism": 0,
    "sexism": 1,
    "neither": 2
}
df['labels'] = df.processed_target.apply(lambda x: label_map[x])
labels = df['label'].astype('int32')
labels_2 = df['snorkel_prob']
tweets = df.text.values

In [ ]:
print(len(df),
len(labels),
len(labels_2),
len(tweets))

4201 4201 4201 4201


In [ ]:
labels, labels_2

(0       0
 1       0
 2       1
 3       0
 4       1
        ..
 4205    1
 4206    0
 4207    0
 4208    0
 4209    1
 Name: label, Length: 4201, dtype: int32, 0       0.549079
 1       0.162369
 2       0.389828
 3       0.162369
 4       0.549079
           ...   
 4205    0.549079
 4206    0.162369
 4207    0.162369
 4208    0.162369
 4209    0.549079
 Name: snorkel_prob, Length: 4201, dtype: float64)

In [ ]:
input_ids = []
for tweet in tweets:

    encoded_sent = tokenizer.encode(tweet, add_special_tokens=True)
    # print(encoded_sent)
    input_ids.append(encoded_sent)


In [ ]:
print(len(input_ids))
print(len(labels))
print(tweets[0])
print(input_ids[0])

4201
4201
RT @Erin_Reimer: In what world would you not be disappointed with that? #MKR
[101, 19387, 1030, 11781, 1035, 24964, 5017, 1024, 1999, 2054, 2088, 2052, 2017, 2025, 2022, 9364, 2007, 2008, 1029, 1001, 12395, 2099, 102]


In [ ]:
max([len(tweet) for tweet in tweets])

152

In [ ]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 256
print(f"Tokenizer shit {tokenizer.pad_token}, {tokenizer.pad_token_id}")

Tokenizer shit [PAD], 0


In [ ]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
input_ids.shape

(4201, 256)

In [ ]:
attn_mask = []

for tweet in input_ids:
    temp_mask = [int(tweet_element > 1) for tweet_element in tweet]
    attn_mask.append(temp_mask) 

In [ ]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=1, test_size=0.2)
train_masks, validation_masks, _,_ = train_test_split(attn_mask, labels, random_state=1, test_size=0.2)
_, _, train_labels_2, validation_labels_2 = train_test_split(input_ids, labels_2, random_state=1, test_size=0.2)

In [ ]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(list(train_labels))
train_labels_2 = torch.tensor(list(train_labels_2))
validation_labels = torch.tensor(list(validation_labels))
validation_labels_2 = torch.tensor(list(validation_labels_2))

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_labels_2)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_labels_2)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False
)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 10
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
# Function to calculate the accuracy of our predictions vs labels

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def metrics(preds, labels):
    predictions = np.argmax(preds, axis=1).flatten()
    true_labels = labels.flatten()
    return f1_score(true_labels, predictions), confusion_matrix(true_labels, predictions)

import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random

# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []
training_loss_values = []
training_acc_values = []
validation_acc_values = []

classifier_2_loss = torch.nn.MSELoss()

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_masks = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_labels_2 = batch[3].to(device)

        model.zero_grad()

        forward_output = model(b_input_ids, 
                               token_type_ids=None,
                               attention_mask=b_input_masks,
                               labels=b_labels)
        
        loss = forward_output[0]

        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            
        
        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    training_loss_values.append(avg_train_loss)




    print("")
    print("Running Validation...")
    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_labels_2 = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    validation_acc_values.append(eval_accuracy/nb_eval_steps)


======== Epoch 1 / 10 ========
Training...
  Batch   100  of    105.    Elapsed: 0:02:18.

  Average training loss: 0.45
  Training epcoh took: 0:02:25

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:15

======== Epoch 2 / 10 ========
Training...
  Batch   100  of    105.    Elapsed: 0:02:29.

  Average training loss: 0.29
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:15

======== Epoch 3 / 10 ========
Training...
  Batch   100  of    105.    Elapsed: 0:02:29.

  Average training loss: 0.18
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:15

======== Epoch 4 / 10 ========
Training...
  Batch   100  of    105.    Elapsed: 0:02:29.

  Average training loss: 0.10
  Training epcoh took: 0:02:36

Running Validation...
  Accuracy: 0.88
  Validation took: 0:00:15

======== Epoch 5 / 10 ========
Training...
  Batch   100  of    105.    Elapsed: 0:02:28.

  Average training loss: 0.0

In [ ]:
from sklearn.metrics import f1_score

true_labels = []
predictions = []

for batch in validation_dataloader:
        
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_labels_2 = batch
    
    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    true_labels.extend(label_ids)
    predictions.extend(logits)
    
    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy
    # Track the number of batches
    nb_eval_steps += 1
    # Report the final accuracy for this validation run.

val_f1, val_CM = metrics(np.array(predictions), np.array(true_labels))
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("-"*30)
print(f"F1 Score = {val_f1}")
print(f"\n\nConfusion matrix :\n\n{val_CM}")

  Accuracy: 0.87
  Validation took: 0:00:30
------------------------------
F1 Score = 0.7308641975308641


Confusion matrix :

[[584  38]
 [ 71 148]]


In [ ]:
# training_loss_values
torch.save(model, "./drive/MyDrive/OSNA Project 2/Model_Basic_10_epochs.h5")